### Loading Libraries

In [1]:
%pip install Levenshtein
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

Note: you may need to restart the kernel to use updated packages.


### Loading Data

In [2]:
sevMil = pd.read_csv("companies_sorted-1.csv")
ft = pd.read_csv("Forbes_top 2000_2022.csv")
db = pd.read_csv("mapping/dbpedia/data/dbpedia_preprocessed_final.csv")

### Creating ID Column for Each Dataset

In [3]:
sevMil = sevMil[['name']]
sevMil = sevMil.applymap(lambda s: s.lower() if type(s) == str else s)
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1M_" + sevMil['id'].astype(str)

In [4]:
ft = ft[['organizationName']]
ft = ft.applymap(lambda s: s.lower() if type(s) == str else s)
ft['id'] =  range(len(ft))
ft['id'] = "FT_ID_" + ft['id'].astype(str)

In [5]:
dbpedia = db[['name']]
dbpedia = dbpedia.applymap(lambda s: s.lower() if type(s) == str else s)
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "dbpedia_" + dbpedia['id'].astype(str)

### Calculating Levenshtein Distance for Gold Standard

#### Between FT and DBPedia Datasets

In [10]:
# Calculate the distance (ratio)
ft_db_lev = pd.DataFrame(product(ft['organizationName'], dbpedia['name']), columns=["ft","dbpedia"])
# Since there are duplicate name, plan to deal with it in Java
ft_db_lev = ft_db_lev.drop_duplicates()
ft_db_lev["LevScore"] = ft_db_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs 
ft_db_lev_filtered = ft_db_lev[ft_db_lev['LevScore'] > 0.8]
ft_db_lev_false = ft_db_lev[ft_db_lev['LevScore'] < 0.3]

# Identify matching and similiar-but-unmatching pairs 
confirmed_true_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] == 1]
edge_cases_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] != 1]
false_cases_ftdb = ft_db_lev_false

# Sample 100 for matching cases, 150 for corner cases, 250 for non-matching cases
true_sample = confirmed_true_ftdb.sample(100, random_state=1)
edge_sample = edge_cases_ftdb.sample(150, random_state=1)
false_sample = false_cases_ftdb.sample(250, random_state=1)

# Goldstandard output
gold_standard_ft_db = pd.concat([true_sample, edge_sample, false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]
gold_standard_ft_db['label'] = gold_standard_ft_db.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ft_db = gold_standard_ft_db.drop_duplicates(subset=['ft', 'dbpedia'], keep='first')
gold_standard_ft_db

,ft_id,db_id,LevScore,ft,dbpedia,label
0,FT_ID_492,dbpedia_5677,1.000000,mitsui fudosan,mitsui fudosan,True
1,FT_ID_1247,dbpedia_2304,1.000000,royal mail,royal mail,True
2,FT_ID_972,dbpedia_2662,1.000000,sasol,sasol,True
3,FT_ID_1224,dbpedia_6456,1.000000,turkish airlines,turkish airlines,True
4,FT_ID_1506,dbpedia_6798,1.000000,hyundai glovis,hyundai glovis,True
...,...,...,...,...,...,...
539,FT_ID_1687,dbpedia_3520,0.250000,doosan,dahabshiil,False
540,FT_ID_1697,dbpedia_2516,0.285714,piraeus financial holdings,insel air,False
541,FT_ID_1341,dbpedia_2145,0.222222,stonex group,navitas limited,False
543,FT_ID_1456,dbpedia_2164,0.173913,unicharm,red back mining,False


In [11]:
gold_standard_ft_db.describe(include = 'all')

,ft_id,db_id,LevScore,ft,dbpedia,label
count,500,500,500.000000,500,500,500
unique,412,469,NaN,412,469,2
top,FT_ID_1163,dbpedia_2630,NaN,cj corporation,bt group,False
freq,7,3,NaN,7,3,400
mean,NaN,NaN,0.553874,NaN,NaN,NaN
std,NaN,NaN,0.358676,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.215734,NaN,NaN,NaN
50%,NaN,NaN,0.551994,NaN,NaN,NaN
75%,NaN,NaN,0.865315,NaN,NaN,NaN


#### Between FT and 7Mil datasets

In [14]:
# Calculate the distance (ratio)
ft_sevM_lev = pd.DataFrame(product(ft['organizationName'], sevMil['name']), columns=["ft","sevMil"])
# Since there are duplicate name, plan to deal with it in Java
ft_sevM_lev = ft_sevM_lev.drop_duplicates()
ft_sevM_lev["LevScore"] = ft_sevM_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs 
ft_sevM_lev_filtered = ft_sevM_lev[ft_sevM_lev['LevScore'] > 0.8]
ft_sevM_lev_false = ft_sevM_lev[ft_sevM_lev['LevScore'] < 0.3]

# Identify matching and similiar-but-unmatching pairs
confirmed_true_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] == 1]
edge_cases_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] != 1]
false_cases_ftsevM = ft_sevM_lev_false

# Sample 100 for matching cases, 150 for corner cases, 250 for non-matching cases
true_sample = confirmed_true_ftsevM.sample(100, random_state=1)
edge_sample = edge_cases_ftsevM.sample(150, random_state=1)
false_sample = false_cases_ftsevM.sample(250, random_state=1)

# Goldstandard output
gold_standard_ftsevM = pd.concat([true_sample, edge_sample, false_sample])
gold_standard_ftsevM = gold_standard_ftsevM.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ftsevM.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM.merge(sevMil, how = 'left', left_on = 'sevMil', right_on = 'name')
gold_standard_ftsevM.rename(columns={'id':'sevM_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM[['ft_id', 'sevM_id', 'LevScore', 'ft', 'sevMil']]
gold_standard_ftsevM['label'] = gold_standard_ftsevM.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ftsevM = gold_standard_ftsevM.drop_duplicates(subset=['ft', 'sevMil'], keep='first')
gold_standard_ftsevM

,ft_id,sevM_id,LevScore,ft,sevMil,label
0,FT_ID_129,7.1M_488,1.000000,intesa sanpaolo,intesa sanpaolo,True
1,FT_ID_1124,7.1M_1551,1.000000,givaudan,givaudan,True
2,FT_ID_524,7.1M_214,1.000000,thales,thales,True
3,FT_ID_553,7.1M_550,1.000000,telus,telus,True
4,FT_ID_1229,7.1M_985,1.000000,alcoa,alcoa,True
...,...,...,...,...,...,...
499,FT_ID_654,7.1M_1367,0.160000,china hongqiao group,optus,False
500,FT_ID_358,7.1M_1559,0.222222,dollar general,cathay pacific airways,False
501,FT_ID_926,7.1M_1128,0.181818,viatris,lyft,False
502,FT_ID_330,7.1M_1846,0.277778,nippon steel,fraser health authority,False


In [15]:
gold_standard_ftsevM.describe(include = 'all')

,ft_id,sevM_id,LevScore,ft,sevMil,label
count,500,500,500.000000,500,500,500
unique,420,415,NaN,420,415,2
top,FT_ID_1940,7.1M_360,NaN,asm international,bmo financial group,False
freq,4,10,NaN,4,10,400
mean,NaN,NaN,0.553169,NaN,NaN,NaN
std,NaN,NaN,0.367845,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.209978,NaN,NaN,NaN
50%,NaN,NaN,0.552403,NaN,NaN,NaN
75%,NaN,NaN,0.889881,NaN,NaN,NaN


Printing data for manually check

In [17]:
gold_standard_ft_db.sort_values('LevScore', ascending = False).to_csv('ft_db_check.csv', index=False)
gold_standard_ftsevM.sort_values('LevScore', ascending = False).to_csv('ft_sevM_check.csv', index=False)

,ft_id,db_id,LevScore,ft,dbpedia,label
0,FT_ID_492,dbpedia_5677,1.0,mitsui fudosan,mitsui fudosan,True
67,FT_ID_362,dbpedia_5902,1.0,hewlett packard enterprise,hewlett packard enterprise,True
80,FT_ID_1273,dbpedia_5821,1.0,dick's sporting goods,dick's sporting goods,True
79,FT_ID_1781,dbpedia_2620,1.0,avnet,avnet,True
78,FT_ID_273,dbpedia_1604,1.0,caixabank,caixabank,True
...,...,...,...,...,...,...
537,FT_ID_1538,dbpedia_6227,0.0,b3,people telecom,False
502,FT_ID_97,dbpedia_184,0.0,ibm,neo solar power,False
496,FT_ID_158,dbpedia_4769,0.0,danaher,wyscout,False
484,FT_ID_1329,dbpedia_2189,0.0,bunzl,aramark,False


Printing Output

In [ ]:
gold_standard_ftsevM.drop(['LevScore','ft','sevMil'], axis=1, inplace=True) 
gold_standard_ft_db.drop(['LevScore','ft','dbpedia'], axis=1, inplace=True)

In [21]:
gold_standard_ft_db.to_csv('data/goldstandard/GS_ft_db.csv', index=False) 
gold_standard_ftsevM.to_csv('data/goldstandard/GS_ft_sevM.csv', index=False)